In [16]:
from sqlalchemy import create_engine
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from dotenv import load_dotenv

In [3]:
load_dotenv()

In [4]:
DB_PARAMS = {
    "dbname": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"), 
    "host": "postgres",
    "port":  5432
}

In [13]:
try:
    db_url = f"postgresql+psycopg2://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}/{DB_PARAMS['dbname']}"
    engine = create_engine(db_url)

    query = """
        SELECT user_id, COUNT(*) AS purchases
        FROM customers
        WHERE event_type = 'purchase'
        GROUP BY user_id
    """
    df = pd.read_sql(query, engine)

except Exception as e:
    print(f"❌ An error occurred: {e}")

In [14]:
X = df[['purchases']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
wcss = []
max_clusters = 10
for i in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

### 1. **Preparación de los datos**
```python
X = df[['purchases']].values
```
- Extrae la columna `purchases` del DataFrame `df` y la convierte en un array NumPy (`.values`).
- `X` contiene los datos que se usarán para el clustering.

```python
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
```
- **`StandardScaler`**: Escala los datos para que tengan media 0 y desviación estándar 1. Esto es importante para que las características tengan la misma escala y no influyan desproporcionadamente en el clustering.
- `X_scaled` contiene los datos escalados.

---

### 2. **Inicialización del método del codo**
```python
wcss = []
max_clusters = 10
```
- **`wcss`**: Lista para almacenar la suma de las distancias cuadradas dentro de los clusters (**Within-Cluster Sum of Squares**, WCSS) para cada número de clusters.
- **`max_clusters`**: Define el número máximo de clusters que se evaluarán (en este caso, de 1 a 10).

---

### 3. **Cálculo del WCSS para diferentes números de clusters**
```python
for i in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)
```
- **Bucle `for`**:
  - Itera sobre diferentes números de clusters (`n_clusters`) desde 1 hasta `max_clusters`.
- **`KMeans`**:
  - Crea un modelo de K-Means con `i` clusters.
  - `random_state=42` asegura que los resultados sean reproducibles.
- **`kmeans.fit(X_scaled)`**:
  - Ajusta el modelo a los datos escalados.
- **`kmeans.inertia_`**:
  - Calcula el WCSS, que mide la suma de las distancias cuadradas entre cada punto y el centroide de su cluster.
  - Este valor se agrega a la lista `wcss`.

---

### 4. **Resultado**
- Al final del bucle, `wcss` contiene los valores del WCSS para cada número de clusters de 1 a `max_clusters`.
- Estos valores se pueden graficar para identificar el "codo" en la curva, que indica el número óptimo de clusters. El "codo" es el punto donde la disminución del WCSS se vuelve menos pronunciada.

Este método ayuda a decidir el número de clusters que mejor agrupa los datos sin sobreajustar.

In [17]:
sns.set_style("darkgrid", rc={"axes.facecolor": "#f0f0f0", "grid.color": "white"})
plt.figure(figsize=(10, 6))
plt.plot(range(1, max_clusters + 1), wcss, linestyle='-', color='darkblue')
plt.title('The Elbow Method', fontsize=16)
plt.xlabel('Number of clusters', fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.show()